In [1]:
import tkinter as tk
from tkinter import messagebox
import os

# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("📝 To-Do List App")
root.geometry("450x500")
root.configure(bg="#2c3e50")
root.resizable(False, False)

TASK_FILE = "tasks.txt"

# ---------------- LOAD TASKS ----------------
def load_tasks():
    if os.path.exists(TASK_FILE):
        with open(TASK_FILE, "r") as file:
            for line in file:
                task_list.insert(tk.END, line.strip())

# ---------------- SAVE TASKS ----------------
def save_tasks():
    with open(TASK_FILE, "w") as file:
        tasks = task_list.get(0, tk.END)
        for task in tasks:
            file.write(task + "\n")

# ---------------- ADD TASK ----------------
def add_task():
    task = task_entry.get()
    if task == "":
        messagebox.showwarning("Warning", "Task cannot be empty!")
    else:
        task_list.insert(tk.END, task)
        task_entry.delete(0, tk.END)
        save_tasks()

# ---------------- DELETE TASK ----------------
def delete_task():
    try:
        selected = task_list.curselection()[0]
        task_list.delete(selected)
        save_tasks()
    except:
        messagebox.showwarning("Warning", "Select a task to delete!")

# ---------------- MARK COMPLETE ----------------
def mark_complete():
    try:
        selected = task_list.curselection()[0]
        task = task_list.get(selected)
        task_list.delete(selected)
        task_list.insert(tk.END, "✔ " + task)
        save_tasks()
    except:
        messagebox.showwarning("Warning", "Select a task!")

# ---------------- CLEAR ALL ----------------
def clear_all():
    confirm = messagebox.askyesno("Confirm", "Delete all tasks?")
    if confirm:
        task_list.delete(0, tk.END)
        save_tasks()

# ---------------- UI DESIGN ----------------
tk.Label(root, text="📝 My To-Do List",
         font=("Arial", 18, "bold"),
         bg="#2c3e50", fg="white").pack(pady=20)

task_entry = tk.Entry(root, font=("Arial", 14), width=25)
task_entry.pack(pady=10)

tk.Button(root, text="Add Task", command=add_task,
          bg="#27ae60", fg="white", width=15).pack(pady=5)

# Frame for Listbox + Scrollbar
frame = tk.Frame(root)
frame.pack(pady=10)

scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

task_list = tk.Listbox(frame, width=40, height=12,
                       yscrollcommand=scrollbar.set,
                       font=("Arial", 12))
task_list.pack()

scrollbar.config(command=task_list.yview)

tk.Button(root, text="Mark Complete", command=mark_complete,
          bg="#2980b9", fg="white", width=15).pack(pady=5)

tk.Button(root, text="Delete Task", command=delete_task,
          bg="#e67e22", fg="white", width=15).pack(pady=5)

tk.Button(root, text="Clear All", command=clear_all,
          bg="#c0392b", fg="white", width=15).pack(pady=5)

load_tasks()

root.mainloop()


In [2]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import json
import os
from datetime import datetime

FILE_NAME = "tasks.json"

# ---------------- LOAD TASKS ----------------
def load_tasks():
    if os.path.exists(FILE_NAME):
        with open(FILE_NAME, "r") as f:
            return json.load(f)
    return []

# ---------------- SAVE TASKS ----------------
def save_tasks():
    with open(FILE_NAME, "w") as f:
        json.dump(tasks, f)

# ---------------- UPDATE LIST ----------------
def update_list():
    task_list.delete(0, tk.END)
    completed = 0

    for task in tasks:
        display = f"{task['title']} | {task['priority']} | {task['date']}"
        if task["done"]:
            display = "✔ " + display
            completed += 1
        task_list.insert(tk.END, display)

    if tasks:
        progress["value"] = (completed / len(tasks)) * 100
    else:
        progress["value"] = 0

# ---------------- ADD TASK ----------------
def add_task():
    title = task_entry.get()
    priority = priority_var.get()
    date = date_entry.get()

    if title == "":
        messagebox.showwarning("Warning", "Task cannot be empty!")
        return

    try:
        datetime.strptime(date, "%Y-%m-%d")
    except:
        messagebox.showwarning("Warning", "Date format must be YYYY-MM-DD")
        return

    tasks.append({
        "title": title,
        "priority": priority,
        "date": date,
        "done": False
    })

    task_entry.delete(0, tk.END)
    save_tasks()
    update_list()

# ---------------- DELETE TASK ----------------
def delete_task():
    try:
        index = task_list.curselection()[0]
        tasks.pop(index)
        save_tasks()
        update_list()
    except:
        messagebox.showwarning("Warning", "Select a task!")

# ---------------- MARK COMPLETE ----------------
def mark_complete():
    try:
        index = task_list.curselection()[0]
        tasks[index]["done"] = not tasks[index]["done"]
        save_tasks()
        update_list()
    except:
        messagebox.showwarning("Warning", "Select a task!")

# ---------------- THEME SWITCH ----------------
def toggle_theme():
    if root["bg"] == "#1e1e1e":
        root.configure(bg="white")
        title_label.config(bg="white", fg="black")
    else:
        root.configure(bg="#1e1e1e")
        title_label.config(bg="#1e1e1e", fg="cyan")

# ---------------- MAIN WINDOW ----------------
root = tk.Tk()
root.title("📝 Advanced To-Do App")
root.geometry("500x600")
root.configure(bg="#1e1e1e")
root.resizable(False, False)

tasks = load_tasks()

title_label = tk.Label(root, text="📝 My Productivity App",
                       font=("Arial", 18, "bold"),
                       bg="#1e1e1e", fg="cyan")
title_label.pack(pady=15)

task_entry = tk.Entry(root, font=("Arial", 14), width=30)
task_entry.pack(pady=5)

priority_var = tk.StringVar(value="Medium")
priority_menu = ttk.Combobox(root, textvariable=priority_var,
                             values=["High", "Medium", "Low"])
priority_menu.pack(pady=5)

date_entry = tk.Entry(root, font=("Arial", 12))
date_entry.insert(0, "YYYY-MM-DD")
date_entry.pack(pady=5)

tk.Button(root, text="Add Task", command=add_task,
          bg="green", fg="white", width=15).pack(pady=5)

task_list = tk.Listbox(root, width=55, height=15)
task_list.pack(pady=10)

tk.Button(root, text="Mark Complete", command=mark_complete,
          bg="blue", fg="white", width=15).pack(pady=5)

tk.Button(root, text="Delete Task", command=delete_task,
          bg="orange", fg="black", width=15).pack(pady=5)

progress = ttk.Progressbar(root, length=400)
progress.pack(pady=10)

tk.Button(root, text="Toggle Theme", command=toggle_theme,
          bg="purple", fg="white", width=15).pack(pady=5)

update_list()

root.mainloop()
